In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [ ]:
#Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Training Dataset

titanic_train = pd.read_csv('/content/drive/My Drive/ML and AI/Assignments/Dataset/Titanic Dataset/train.csv')
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#Check for null values

titanic_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [ ]:
# check the columns in the dataset

titanic_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
#Preprocess the data, Convert the String columns in to numbers

label_encoder = preprocessing.LabelEncoder()
titanic_train['Sex'] = label_encoder.fit_transform(titanic_train['Sex'])
titanic_train['Embarked'] = label_encoder.fit_transform(titanic_train['Embarked'])

In [ ]:
#Identify significant columns by using Random Forest

features = ['Sex', 'Pclass', 'SibSp', 'Embarked', 'Age', 'Fare']
rf_model = RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)
rf_model.fit(X=titanic_train[features], y=titanic_train['Survived'])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#Print the OOB accuracy value

print("OOB Accuracy is ", rf_model.oob_score_)

OOB Accuracy is  0.8087739032620922


In [ ]:
#Identify the important features

for feature, imp in zip (features, rf_model.feature_importances_):
  print(feature, imp)

Sex 0.27223292868687604
Pclass 0.08879734519285015
SibSp 0.05218627953557116
Embarked 0.033567211655280774
Age 0.2693534225319032
Fare 0.28386281239751865


In [ ]:
#Initialize Decision tree classifier

tree_model = tree.DecisionTreeClassifier(max_depth=8)

In [ ]:
#Build Model

predictors = pd.DataFrame([titanic_train['Sex'], titanic_train['Age'], titanic_train['Fare']]).T
tree_model.fit(X=predictors, y=titanic_train['Survived'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
#Generate Dot file so that you can visually see the decision tree

with open("Dtree.dot", 'w') as f:
  f = tree.export_graphviz(tree_model, feature_names=['Sex','Age','Fare'], out_file=f)

In [ ]:
#Check your model's score

tree_model.score(X=predictors, y=titanic_train['Survived'])

0.8706411698537683

In [ ]:
#Peediction for the test data begins
#Import the test data

titanic_test = pd.read_csv('/content/drive/My Drive/ML and AI/Assignments/Dataset/Titanic Dataset/test.csv')
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [ ]:
#Preprocessing for test data. Check if any missing values available

titanic_test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [ ]:
#Preprocessing for test data. Conver the String features into Numbers

titanic_test['Sex'] = label_encoder.fit_transform(titanic_test['Sex'])

In [ ]:
#Create test features with required columns

test_features = pd.DataFrame([titanic_test['Sex'], titanic_test['Age'], titanic_test['Fare']]).T

In [ ]:
#Test the data with our model

test_preds = tree_model.predict(test_features)
predicted_output = pd.DataFrame({"PassengerId":titanic_test["PassengerId"], "Survived":test_preds})
predicted_output.head


<bound method NDFrame.head of      PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
412         1305         0
413         1306         1
414         1307         0
415         1308         0
416         1309         0

[417 rows x 2 columns]>

In [58]:
#Save the predicted output to a csv

predicted_output.to_csv("/content/drive/My Drive/ML and AI/Colab Notebooks/Day24/Dtree_titanic_predictedOutput.csv", index=False)